## Video steganography (Kaggle-ready)

- **Inputs**: videos in `/kaggle/input/<your-dataset>/...` (auto-detected)
- **Outputs**: saved to `/kaggle/working/` (models + demo files)

If no video dataset is attached, the notebook falls back to a small dummy dataset so it still runs.

In [1]:
# Core
import os
import glob
import random
from dataclasses import dataclass

import numpy as np
import cv2

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm.auto import tqdm
import torchvision
from torchvision import transforms

# Crypto (AES-GCM)
try:
    from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
    from cryptography.hazmat.backends import default_backend
except ImportError as e:
    raise ImportError(
        "Missing dependency: cryptography. In Kaggle: Settings → Internet ON, then `pip install cryptography`."
    ) from e


def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True


def get_device() -> torch.device:
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


@dataclass
class CFG:
    seed: int = 42

    # Video
    frames: int = 16
    video_size: int = 64  # H=W after resizing

    # Cover image for stego
    cover_size: int = 256

    # Model
    latent_dim: int = 256

    # Training
    batch_size: int = 4
    ae_epochs: int = 2
    stego_epochs: int = 2
    lr: float = 1e-3

    # Secret bits we ask the stego nets to learn (<= cover_size*cover_size)
    secret_bits_len: int = 8192

    # Kaggle paths
    kaggle_input_root: str = "/kaggle/input"
    work_dir: str = "/kaggle/working"

    @property
    def models_dir(self) -> str:
        return os.path.join(self.work_dir, "models")

    @property
    def out_dir(self) -> str:
        return os.path.join(self.work_dir, "outputs")


def find_video_files(root: str, exts=(".mp4", ".avi", ".mov", ".mkv")):
    if not root or not os.path.exists(root):
        return []
    files = []
    for ext in exts:
        files.extend(glob.glob(os.path.join(root, "**", f"*{ext}"), recursive=True))
    return sorted(files)


def autodetect_video_root(input_root: str = "/kaggle/input") -> str | None:
    if not os.path.exists(input_root):
        return None
    candidates = [os.path.join(input_root, d) for d in os.listdir(input_root)]
    candidates = [d for d in candidates if os.path.isdir(d)]
    for d in candidates:
        if len(find_video_files(d)) > 0:
            return d
    return None


cfg = CFG()
seed_everything(cfg.seed)
device = get_device()
print("Device:", device)
print("Models dir:", cfg.models_dir)
print("Outputs dir:", cfg.out_dir)

In [2]:
def extract_frames(video_path, max_frames=None, resize_dim=(128, 128)):
    """
    Extracts frames from a video file, resizes them, and normalizes them.
    Returns a PyTorch tensor of shape (C, T, H, W) where T is the number of frames.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if resize_dim is not None:
            frame = cv2.resize(frame, resize_dim)

        frames.append(frame)
        if max_frames is not None and len(frames) >= max_frames:
            break

    cap.release()
    if not frames:
        raise ValueError(f"Could not extract any frames from {video_path}")

    frames_np = np.array(frames).astype(np.float32) / 255.0
    tensor_frames = torch.from_numpy(frames_np).permute(3, 0, 1, 2)
    return tensor_frames

In [ ]:
def compile_video(frames_tensor, output_path, fps=30):
    """
    Reconstructs a video from a PyTorch tensor of shape (C, T, H, W).
    """
    if frames_tensor.requires_grad:
        frames_tensor = frames_tensor.detach()
    frames_tensor = frames_tensor.cpu()

    frames_np = frames_tensor.permute(1, 2, 3, 0).numpy()
    frames_np = np.clip(frames_np * 255.0, 0, 255).astype(np.uint8)

    T, H, W, C = frames_np.shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (W, H))

    for i in range(T):
        frame = cv2.cvtColor(frames_np[i], cv2.COLOR_RGB2BGR)
        out.write(frame)

    out.release()

# **Video auto encoder**


In [ ]:
# (imports moved to the top Kaggle setup cell)

In [ ]:
class VideoEncoder(nn.Module):
    def __init__(self, in_channels=3, latent_dim=256):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, 32, 3, 2, 1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.Conv3d(32, 64, 3, 2, 1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.Conv3d(64, 128, 3, 2, 1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.Conv3d(128, 256, 3, 2, 1),
            nn.BatchNorm3d(256),
            nn.ReLU(),
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(4096, latent_dim)

    def forward(self, x):
        return self.fc(self.flatten(self.encoder(x)))

In [ ]:
class VideoDecoder(nn.Module):
    def __init__(self, out_channels=3, latent_dim=256):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 4096)
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(256, 128, 3, 2, 1, 1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.ConvTranspose3d(128, 64, 3, 2, 1, 1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 32, 3, 2, 1, 1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.ConvTranspose3d(32, out_channels, 3, 2, 1, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(self.fc(x).view(-1, 256, 1, 4, 4))

In [ ]:
class VideoAutoencoder(nn.Module):
    def __init__(self, in_channels=3, latent_dim=256):
        super().__init__()
        self.encoder = VideoEncoder(in_channels, latent_dim)
        self.decoder = VideoDecoder(in_channels, latent_dim)

    def forward(self, x):
        latent = self.encoder(x)
        return self.decoder(latent), latent

# **Encryption**

In [ ]:
# (imports moved to the top Kaggle setup cell)

In [ ]:
def generate_key():
    return os.urandom(32)

In [ ]:
def generate_iv():
    return os.urandom(12)

In [ ]:
def tensor_to_bytes(tensor):
    tensor_np = tensor.cpu().detach().numpy().astype("float32")
    return tensor_np.tobytes(), tensor_np.shape

In [ ]:
def bytes_to_tensor(byte_data, shape, device="cpu"):
    import numpy as np

    tensor_np = np.frombuffer(byte_data, dtype="float32").copy().reshape(shape)
    return torch.from_numpy(tensor_np).to(device)

In [ ]:
def encrypt_data(data_bytes, key, iv):
    encryptor = Cipher(
        algorithms.AES(key), modes.GCM(iv), backend=default_backend()
    ).encryptor()
    return encryptor.update(data_bytes) + encryptor.finalize(), encryptor.tag



In [ ]:
def decrypt_data(ciphertext, tag, key, iv):
    decryptor = Cipher(
        algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend()
    ).decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()

In [ ]:
class LatentEncryptor:
    def __init__(self, key=None):
        self.key = key if key else generate_key()

    def encrypt(self, latent_tensor):
        data_bytes, shape = tensor_to_bytes(latent_tensor)
        iv = generate_iv()
        ciphertext, tag = encrypt_data(data_bytes, self.key, iv)
        return ciphertext, {"iv": iv, "tag": tag, "shape": shape}

    def decrypt(self, ciphertext, metadata, device="cpu"):
        plaintext = decrypt_data(ciphertext, metadata["tag"], self.key, metadata["iv"])
        return bytes_to_tensor(plaintext, metadata["shape"], device=device)

In [ ]:
def ciphertext_to_bits(ciphertext, max_len=None):
    import numpy as np

    byte_array = np.frombuffer(ciphertext, dtype=np.uint8)
    bit_array = np.unpackbits(byte_array).astype(np.float32)
    if max_len is not None:
        padded = np.zeros(max_len, dtype=np.float32)
        padded[: len(bit_array)] = bit_array
        bit_array = padded
    return torch.from_numpy(bit_array)

In [ ]:
def bits_to_ciphertext(bit_tensor, original_byte_len):
    import numpy as np

    bit_array = (bit_tensor.cpu().numpy() >= 0.5).astype(np.uint8)[
        : original_byte_len * 8
    ]
    return np.packbits(bit_array).tobytes()

# **# Image Generation**

In [ ]:
# (imports moved to the top Kaggle setup cell)

In [ ]:
class ImageGenerator:
    """Wrapper for AI image generation."""

    def __init__(self, device="cpu", use_dummy=True):
        self.device = device
        self.use_dummy = use_dummy
        self.pipeline = None

        if not self.use_dummy:
            try:
                from diffusers import StableDiffusionPipeline

                self.pipeline = StableDiffusionPipeline.from_pretrained(
                    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
                )
                self.pipeline = self.pipeline.to(self.device)
            except ImportError:
                print("Diffusers not installed. Falling back to dummy generator.")
                self.use_dummy = True

    def generate_cover(
        self,
        prompt="A beautiful realistic landscape photo, 4k resolution",
        size=(256, 256),
    ):
        if self.use_dummy or self.pipeline is None:
            img_tensor = (
                torch.rand((3, size[0], size[1]), dtype=torch.float32)
                .to(self.device)
                .unsqueeze(0)
            )
            import torch.nn.functional as F

            img_tensor = F.avg_pool2d(
                img_tensor, kernel_size=5, stride=1, padding=2
            ).squeeze(0)
            return (img_tensor - img_tensor.min()) / (
                img_tensor.max() - img_tensor.min() + 1e-8
            )
        else:
            image = self.pipeline(
                prompt, height=size[0], width=size[1], num_inference_steps=20
            ).images[0]
            image_np = np.array(image).astype(np.float32) / 255.0
            return torch.from_numpy(image_np).permute(2, 0, 1).to(self.device)




# **Stego Networks**

In [ ]:
# (imports moved to the top Kaggle setup cell)

In [ ]:
class HiderNetwork(nn.Module):
    def __init__(self, cover_channels=3, secret_channels=1, hidden_channels=64):
        super().__init__()
        in_channels = cover_channels + secret_channels
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, cover_channels, 3, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, cover, secret):
        return self.net(torch.cat([cover, secret], dim=1))

In [ ]:
class RevealerNetwork(nn.Module):
    def __init__(self, stego_channels=3, secret_channels=1, hidden_channels=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(stego_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, secret_channels, 3, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, stego):
        return self.net(stego)

In [ ]:
def format_secret_for_hiding(secret_bits, target_shape):
    B, C, H, W = target_shape
    total_elements = C * H * W
    padded = torch.zeros(B, total_elements, device=secret_bits.device)
    for i in range(B):
        seq = secret_bits[i] if secret_bits.dim() > 1 else secret_bits
        length = min(len(seq), total_elements)
        padded[i, :length] = seq[:length]
    return padded.view(B, C, H, W)

In [ ]:
def extract_secret_from_prediction(secret_pred_spatial, original_length):
    return secret_pred_spatial.view(secret_pred_spatial.shape[0], -1)[
        :, :original_length
    ]

# **Training**

In [ ]:
# (imports moved to the top Kaggle setup cell)

In [ ]:
# Everything is defined in this notebook (no local .py imports needed on Kaggle).

In [ ]:
class DummyVideoDataset(Dataset):
    def __init__(self, num_samples=100, frames=16, height=64, width=64):
        self.num_samples = num_samples
        self.frames = frames
        self.height = height
        self.width = width

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return torch.rand(
            (3, self.frames, self.height, self.width), dtype=torch.float32
        )

In [ ]:
# (imports moved to the top Kaggle setup cell)

In [ ]:
class RealVideoDataset(Dataset):
    """Loads actual .mp4 or .avi videos from a directory for autoencoder training."""
    def __init__(self, directory, frames=16, height=64, width=64):
        self.video_paths = glob.glob(os.path.join(directory, "**", "*.avi"), recursive=True) + \
                           glob.glob(os.path.join(directory, "**", "*.mp4"), recursive=True)
        self.frames = frames
        self.height = height
        self.width = width
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((self.height, self.width)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        cap = cv2.VideoCapture(self.video_paths[idx])
        frames = []
        while len(frames) < self.frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_tensor = self.transform(frame) # shape (3, H, W)
            frames.append(frame_tensor)
        cap.release()
        
        # If video is too short, pad it with the last frame
        while len(frames) < self.frames and len(frames) > 0:
            frames.append(frames[-1])
            
        # If video couldn't be loaded at all, return zeros (edge case fallback)
        if len(frames) == 0:
            return torch.zeros((3, self.frames, self.height, self.width), dtype=torch.float32)
            
        # Stack into (C, F, H, W)
        video_tensor = torch.stack(frames, dim=1)
        return video_tensor

In [ ]:
def train_video_autoencoder(model, dataloader, *, epochs: int, device, lr: float, save_dir: str):
    print("--- Training Video Autoencoder ---")
    if dataloader is None or len(dataloader) == 0:
        print("No data found for autoencoder training; skipping.")
        return

    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running = 0.0
        pbar = tqdm(dataloader, desc=f"AE epoch {epoch+1}/{epochs}", leave=False)
        for batch in pbar:
            batch = batch.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            reconstructed, _ = model(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            running += loss.item()
            pbar.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}/{epochs} | loss={running/len(dataloader):.6f}")

    os.makedirs(save_dir, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(save_dir, "video_autoencoder.pth"))

In [ ]:
def train_stego_networks(
    hider,
    revealer,
    image_generator,
    *,
    epochs: int,
    device,
    lr: float,
    secret_bits_len: int,
    cover_size: int,
    save_dir: str,
    batch_size: int = 4,
    iterations_per_epoch: int = 50,
    img_weight: float = 10.0,
):
    print("\n--- Training Steganography Networks ---")
    hider.to(device)
    revealer.to(device)

    criterion_mse = nn.MSELoss()
    criterion_bce = nn.BCELoss()
    optimizer = optim.Adam(list(hider.parameters()) + list(revealer.parameters()), lr=lr)

    for epoch in range(epochs):
        hider.train()
        revealer.train()
        img_loss_sum = 0.0
        bit_loss_sum = 0.0

        for _ in tqdm(range(iterations_per_epoch), desc=f"Stego epoch {epoch+1}/{epochs}", leave=False):
            covers = torch.stack(
                [image_generator.generate_cover(size=(cover_size, cover_size)) for _ in range(batch_size)]
            ).to(device)

            secret_bits = torch.randint(0, 2, (batch_size, secret_bits_len), device=device).float()
            spatial_secret = format_secret_for_hiding(secret_bits, (batch_size, 1, cover_size, cover_size))

            stego = hider(covers, spatial_secret)
            secret_pred = extract_secret_from_prediction(revealer(stego), secret_bits_len)
            secret_pred = torch.clamp(secret_pred, 1e-6, 1.0 - 1e-6)

            l_img = criterion_mse(stego, covers)
            l_bit = criterion_bce(secret_pred, secret_bits)
            loss = (img_weight * l_img) + l_bit

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            img_loss_sum += l_img.item()
            bit_loss_sum += l_bit.item()

        print(
            f"Epoch {epoch+1}/{epochs} | img_loss={img_loss_sum/iterations_per_epoch:.6f} | bit_loss={bit_loss_sum/iterations_per_epoch:.6f}"
        )

    os.makedirs(save_dir, exist_ok=True)
    torch.save(hider.state_dict(), os.path.join(save_dir, "hider.pth"))
    torch.save(revealer.state_dict(), os.path.join(save_dir, "revealer.pth"))

In [ ]:
# =============================
# Kaggle: train models
# =============================

os.makedirs(cfg.models_dir, exist_ok=True)
os.makedirs(cfg.out_dir, exist_ok=True)

video_root = autodetect_video_root(cfg.kaggle_input_root)
video_files = find_video_files(video_root) if video_root else []

if len(video_files) > 0:
    print(f"Found {len(video_files)} videos under: {video_root}")
    video_dataset = RealVideoDataset(
        directory=video_root,
        frames=cfg.frames,
        height=cfg.video_size,
        width=cfg.video_size,
    )
else:
    print("No videos found in /kaggle/input. Using a dummy dataset so the notebook runs.")
    video_dataset = DummyVideoDataset(
        num_samples=32,
        frames=cfg.frames,
        height=cfg.video_size,
        width=cfg.video_size,
    )

pin = device.type == "cuda"
video_loader = DataLoader(
    video_dataset,
    batch_size=cfg.batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=pin,
    drop_last=True,
)

# 1) Video autoencoder
video_ae = VideoAutoencoder(in_channels=3, latent_dim=cfg.latent_dim)
train_video_autoencoder(
    video_ae,
    video_loader,
    epochs=cfg.ae_epochs,
    device=device,
    lr=cfg.lr,
    save_dir=cfg.models_dir,
)

# 2) Stego networks (cover images are generated; keep dummy generator for Kaggle stability)
hider = HiderNetwork(cover_channels=3, secret_channels=1, hidden_channels=32)
revealer = RevealerNetwork(stego_channels=3, secret_channels=1, hidden_channels=32)
img_gen = ImageGenerator(device=device, use_dummy=True)

train_stego_networks(
    hider,
    revealer,
    img_gen,
    epochs=cfg.stego_epochs,
    device=device,
    lr=cfg.lr,
    secret_bits_len=cfg.secret_bits_len,
    cover_size=cfg.cover_size,
    save_dir=cfg.models_dir,
    batch_size=cfg.batch_size,
)

print("Training complete. Checkpoints saved to:", cfg.models_dir)

# **Pipeline**

In [ ]:
# (imports moved to the top Kaggle setup cell)

In [ ]:
# Everything is defined in this notebook (no local .py imports needed on Kaggle).

In [ ]:
def save_image(tensor, path):
    import torchvision

    torchvision.utils.save_image(tensor, path)

In [ ]:
class SteganoPipeline:
    def __init__(
        self,
        *,
        device,
        models_dir: str,
        frames: int,
        video_size: int,
        cover_size: int,
        latent_dim: int,
        use_dummy_covers: bool = True,
    ):
        self.device = device
        self.frames = frames
        self.video_size = video_size
        self.cover_size = cover_size

        self.autoencoder = VideoAutoencoder(in_channels=3, latent_dim=latent_dim).to(device)
        self.hider = HiderNetwork(cover_channels=3, secret_channels=1).to(device)
        self.revealer = RevealerNetwork(stego_channels=3, secret_channels=1).to(device)

        # Runtime-only secret key (keep same object for hide+extract)
        self.encryptor = LatentEncryptor()

        # Covers (keep dummy by default for Kaggle reliability)
        self.generator = ImageGenerator(device=device, use_dummy=use_dummy_covers)

        # Load checkpoints if present
        ae_ckpt = os.path.join(models_dir, "video_autoencoder.pth")
        hider_ckpt = os.path.join(models_dir, "hider.pth")
        revealer_ckpt = os.path.join(models_dir, "revealer.pth")

        if os.path.exists(ae_ckpt):
            self.autoencoder.load_state_dict(torch.load(ae_ckpt, map_location=device))
        if os.path.exists(hider_ckpt):
            self.hider.load_state_dict(torch.load(hider_ckpt, map_location=device))
        if os.path.exists(revealer_ckpt):
            self.revealer.load_state_dict(torch.load(revealer_ckpt, map_location=device))

        self.autoencoder.eval()
        self.hider.eval()
        self.revealer.eval()

    def hide_video(self, video_path: str, output_image_path: str):
        print(f"1. Extracting frames from {video_path}...")
        frames = extract_frames(
            video_path,
            max_frames=self.frames,
            resize_dim=(self.video_size, self.video_size),
        ).unsqueeze(0).to(self.device)

        print("2. Compressing video into latent vector...")
        with torch.no_grad():
            _, latent = self.autoencoder(frames)

        print("3. Encrypting latent vector using AES-GCM...")
        ciphertext, metadata = self.encryptor.encrypt(latent[0])

        print(f"4. Generating cover image ({self.cover_size}x{self.cover_size})...")
        cover_image = self.generator.generate_cover(size=(self.cover_size, self.cover_size)).unsqueeze(0).to(self.device)

        print("5. Packing encrypted data into spatial tensor...")
        max_bits = self.cover_size * self.cover_size
        spatial_secret = format_secret_for_hiding(
            ciphertext_to_bits(ciphertext, max_bits).unsqueeze(0).to(self.device),
            (1, 1, self.cover_size, self.cover_size),
        )

        print("6. Embedding secret into cover image...")
        with torch.no_grad():
            stego_image = self.hider(cover_image, spatial_secret)

        print(f"7. Saving stego image to {output_image_path}...")
        save_image(stego_image[0], output_image_path)
        return metadata, len(ciphertext)

    def extract_video(self, stego_image_path: str, metadata, cipher_len: int, output_video_path: str):
        from torchvision.io import read_image

        print(f"1. Loading stego image from {stego_image_path}...")
        stego_image = (read_image(stego_image_path).float() / 255.0).unsqueeze(0).to(self.device)

        print("2. Extracting spatial data...")
        with torch.no_grad():
            secret_pred_spatial = self.revealer(stego_image)

        print("3. Reconstructing bit stream...")
        bit_tensor = extract_secret_from_prediction(secret_pred_spatial, cipher_len * 8)

        print("4. Repacking bits to ciphertext...")
        recovered_ciphertext = bits_to_ciphertext(bit_tensor[0], cipher_len)

        print("5. Decrypting latent vector using AES-GCM...")
        recovered_latent = self.encryptor.decrypt(recovered_ciphertext, metadata, device=self.device)

        print("6. Reconstructing video frames...")
        with torch.no_grad():
            reconstructed_frames = self.autoencoder.decoder(recovered_latent.unsqueeze(0))

        print(f"7. Saving reconstructed video to {output_video_path}...")
        compile_video(reconstructed_frames[0], output_video_path, fps=15)
        print("--- Decode complete ---")

In [ ]:
# =============================
# Kaggle: quick end-to-end demo
# =============================

os.makedirs(cfg.out_dir, exist_ok=True)

# Use a real video if available, else create a tiny dummy
video_root = autodetect_video_root(cfg.kaggle_input_root)
video_files = find_video_files(video_root) if video_root else []

if len(video_files) > 0:
    demo_video_path = video_files[0]
    print("Using demo video:", demo_video_path)
else:
    demo_video_path = os.path.join(cfg.out_dir, "dummy_video.mp4")
    print("No input videos found; writing dummy video:", demo_video_path)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(demo_video_path, fourcc, 15, (cfg.video_size, cfg.video_size))
    for _ in range(cfg.frames):
        out.write(np.random.randint(0, 255, (cfg.video_size, cfg.video_size, 3), dtype=np.uint8))
    out.release()

pipeline = SteganoPipeline(
    device=device,
    models_dir=cfg.models_dir,
    frames=cfg.frames,
    video_size=cfg.video_size,
    cover_size=cfg.cover_size,
    latent_dim=cfg.latent_dim,
    use_dummy_covers=True,
)

stego_img_path = os.path.join(cfg.out_dir, "stego_output.png")
recon_video_path = os.path.join(cfg.out_dir, "reconstructed_video.mp4")

metadata, cipher_length = pipeline.hide_video(demo_video_path, stego_img_path)
pipeline.extract_video(stego_img_path, metadata, cipher_length, recon_video_path)

print("Demo outputs written to:", cfg.out_dir)

